In [ ]:
%%capture
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext training_ml_control
%set_random_seed 12

In [ ]:
%presentation_style

In [ ]:
%load_latex_macros

In [ ]:
%autoreload
import os
import warnings
from dataclasses import dataclass
from typing import Protocol

import casadi
import do_mpc
import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from gymnasium import Env
from IPython.display import HTML
from ipywidgets import interact, widgets
from matplotlib.animation import FuncAnimation
from numpy.typing import NDArray

from training_ml_control.environment import (
    create_inverted_pendulum_environment,
)

from training_ml_control.shortest_path_problem import (
    create_shortest_path_graph,
    plot_shortest_path_graph,
    plot_all_paths_graph,
)

from training_ml_control.plots import (
    create_inverted_pendulum_environment,
    plot_inverted_pendulum_results,
    animate_mass_spring_damper_simulation,
    animate_inverted_pendulum_simulation,
    animate_full_inverted_pendulum_simulation,
)

from training_ml_control.nb_utils import (
    display_array,
    show_video,
)

warnings.simplefilter("ignore", UserWarning)
sns.set_theme()
plt.rcParams["figure.figsize"] = [9, 5]
# This is needed because inside docker the rendering of mujoco environments may not work.
render_mode = "rgb_array" if os.environ.get("DISPLAY") else None

:::{figure} ./_static/images/aai-institute-cover.png
:width: 90%
:align: center
---
name: aai-institute
---
:::

Unfortunately, the analytically convenient linear quadratic problem formulations are often not satisfactory. There are two main reasons for this:

- The system may be nonlinear, and it may be inappropriate to use for
  control purposes. Moreover, some of the control variables may be naturally
  discrete, and this is incompatible with the linear system viewpoint.
  
- There may be control and/or state constraints, which are not handled
  adequately through quadratic penalty terms in the cost function. For
  example, the motion of a car may be constrained by the presence of
  obstacles and hardware limitations.
  The solution obtained from a linear quadratic model may not be suitable for such
  a problem, because quadratic penalties treat constraints "softly"
  and may produce trajectories that violate the constraints.

Model Predictive Control (MPC) is a control algorithm based on solving an **on-line** optimal control problem. A receding horizon approach is used which can be summarized in the following steps:

1. At time $t$ and for the current state $x_t$, solve, on-line, an open-loop optimal control problem over some future interval taking account the current and future constraints.
2. Apply the first step in the optimal control sequence.
3. Repeat the procedure at time $t + 1$ using the current state $x_{t + 1}$.

:::{figure} _static/images/30_model_predictive_control_horizon.svg
Illustration of the problem solved by MPC at state $x_k$.
We minimize the cost function over the next $l$ stages.
We then apply the control of the optimizing sequence up to the control horizon.
In most cases, the control horizon is set to 1.
:::

:::{figure} _static/images/30_mpc_block_diagram.svg
MPC Block Diagram.
:::

A major difference between MPC and finite-state stochastic control problems
that are popular in the RL/artificial intelligence literature is that in MPC
the state and control spaces are continuous/infinite, such as for example
in self-driving cars, the control of aircraft and drones, or the operation of
chemical processes

## Iterative Linear Quadratic Regulator

Iterative Linear Quadratic Regulator (iLQR) is an extension of LQR control to non-linear system with non-linear quadratic costs.

The idea is to approximate the cost and dynamics as quadratic and affine, respectively, then exactly solve the resulting LQR problem.

Given a discrete-time non-linear system with state-space representation: 

$$
\mathbf{x}_{t+1} = f(\mathbf{x}_t, \mathbf{u}_t)
$$

we approximate it with a first-order Taylor-series expansion about nominal trajectories
$\mathbf{X} = \{\mathbf{x}_0, \dots, \mathbf{x}_N\}, \mathbf{U} = \{ \mathbf{u}_0, \dots, \mathbf{u}_N \}$:

$$
\begin{array}{lll}
\mathbf{x}_{t+1} + \delta\mathbf{x}_{t+1} &= f(\mathbf{x}_t + \delta \mathbf{x}_t, \mathbf{u}_t + \delta \mathbf{u}_t) \\ 
& \approx f(\mathbf{x}_t, \mathbf{u}_t)
+ \frac{\partial f}{\partial \mathbf{x}}|_{\mathbf{x}_t, \mathbf{u}_t}\delta\mathbf{x}_t
+ \frac{\partial f}{\partial \mathbf{u}}|_{\mathbf{x}_t, \mathbf{u}_t}\delta\mathbf{u}_t\\
\delta\mathbf{x}_{t+1} &= A_t\delta\mathbf{x}_t + B_t\delta\mathbf{u}_t
\end{array}
$$

with
$A_t = A(\mathbf{x}_t, \mathbf{u}_t) = \frac{\partial f}{\partial \mathbf{x}}|_{\mathbf{x}_t, \mathbf{u}_t}$ and
$B_t = B(\mathbf{x}_t, \mathbf{u}_t) = \frac{\partial f}{\partial \mathbf{u}}|_{\mathbf{x}_t, \mathbf{u}_t}$

Given a general cost function that is not linear-quadratic, we use a second-order Taylor Series Expansion to linearize the dynamics into a form common for optimal control problems:

$$
\begin{array}{lll}
J_N(\mathbf{x}_0, \mathbf{U}) &=
l_f(\mathbf{x}_N) + \sum \limits_{t=1}^{N-1} l(\mathbf{x}_t, \mathbf{u}_t)\\
&\approx \frac{1}{2} \mathbf{x}^T_N Q \mathbf{x}_N + q_N^T \mathbf{x}_N
+ \sum \limits_{t=1}^{N-1}
\frac{1}{2} \mathbf{x}^T_t Q_t \mathbf{x}_t
+ \frac{1}{2} \mathbf{u}^T_t R_t \mathbf{u}_t
+ \frac{1}{2} \mathbf{x}^T_t H_t \mathbf{u}_t
+ \frac{1}{2} \mathbf{u}^T_t H_t^T \mathbf{x}_t
+ q_t^T \mathbf{x}_t +  + r_t^T \mathbf{u}_t
\end{array}
$$

We define the following variables for convenience:

$$
\begin{array}{lll}
l_x &:= \frac{\partial l}{\partial \mathbf{x}}|_{\mathbf{x}_t, \mathbf{u}_t} = Q_t \mathbf{x}_t + q_t
\\
l_u &:= \frac{\partial l}{\partial \mathbf{u}}|_{\mathbf{x}_t, \mathbf{u}_t} = R_t \mathbf{u}_t + r_t
\\
l_{xx} &:= \frac{\partial^2 l}{\partial \mathbf{x}^2}|_{\mathbf{x}_t, \mathbf{u}_t} = Q_t
\\
l_{uu} &:= \frac{\partial^2 l}{\partial \mathbf{u}^2}|_{\mathbf{x}_t, \mathbf{u}_t} = R_t
\\
l_{xu} &:= \frac{\partial^2 l}{\partial \mathbf{x}\mathbf{u}}|_{\mathbf{x}_t, \mathbf{u}_t} = H_t
\\
l_{ux} &:= \frac{\partial^2 l}{\partial \mathbf{u}\mathbf{x}}|_{\mathbf{x}_t, \mathbf{u}_t} = H_t^T
\end{array}
$$

We can apply Bellman's Principle of Optimality to define the optimal cost-to-go:

$$
\begin{array}{lll}
V_N(\mathbf{x}_N) &= g_f(\mathbf{x}_N)\\
V_{t}(\mathbf{x}_t) &= \displaystyle \min_u {g(\mathbf{x}_{t}, \mathbf{u}_{t}) + V_{t+1}(f(\mathbf{x}_{t}, \mathbf{u}_{N-t}))}\\
V_{t}(\mathbf{x}_t) &= \displaystyle \min_u Q_{t}(\mathbf{x}_{t}, \mathbf{u}_{t})
\end{array}\\
$$

The $Q$-function is the discrete-time analogue of the Hamiltonian, sometimes known as the pseudo-Hamiltonian.

We approximate the cost-to-go function as locally quadratic near the nominal trajectory gives us (we drop the time index in some of the equations for the sake of readability):

$$
\delta V(\mathbf{x}) = s^T \delta\mathbf{x} + \frac{1}{2} \delta\mathbf{x}^T S \delta\mathbf{x}
$$

with 
$s = \frac{\partial V}{\partial \mathbf{x}}|_{\mathbf{x}},
S = \frac{\partial^2 V}{\partial \mathbf{x}^2}|_{\mathbf{x}}$

Similarily:

$$
\delta Q(\mathbf{x}, \mathbf{u}) = 
\frac{1}{2}
\begin{bmatrix} \delta\mathbf{x} \\ \delta\mathbf{u} \end{bmatrix}^T
\begin{bmatrix} Q_{xx} & Q_{xu} \\ Q_{ux} & Q_{uu} \end{bmatrix}
\begin{bmatrix} \delta\mathbf{x} \\ \delta\mathbf{u} \end{bmatrix}
+
\begin{bmatrix} Q_{x} \\ Q_{u} \end{bmatrix}^T
\begin{bmatrix} \delta\mathbf{x} \\ \delta\mathbf{u} \end{bmatrix}
$$

with:

$$
\begin{array}{lll}
Q_x &= l_x + s_{t+1} A_t \\
Q_u &= l_u + s_{t+1} B_t \\
Q_{xx} &= l_{xx} + A_t^T S_{t+1} A_t \\
Q_{uu} &= l_{uu} + B_t^T S_{t+1} B_t \\
Q_{ux} &= l_{ux} + B_t^T S_{t+1} A_t
\end{array}
$$

The optimal control modification $\delta\mathbf{u}^∗$ for some state perturbation $\delta\mathbf{x}$, is obtained by minimizing the quadratic model:

$$
\delta\mathbf{u}^∗(\delta\mathbf{x}) = \displaystyle\arg\min_{\delta\mathbf{u}} Q(\delta\mathbf{x}, \delta\mathbf{u}) = k + K\delta\mathbf{x}
$$

This is a locally-linear feedback policy with:

$$
k := -Q_{uu}^{-1}Q_u\\
K := -Q_{uu}^{-1}Q_{ux}
$$

Plugging this back into the expansion of $Q$, a quadratic model of $V$ is obtained. After simplification it is:

$$
\begin{array}{lll}
\Delta V &= \frac{1}{2} k^T Q_{uu} k + k^T Q_u\\
s &= Q_x + K^T Q_{uu} k + K^T Q_u + Q_{ux}^T k\\
S &= Q_{xx} + K^T Q_{uu} K + K^T Q_{ux} + Q_{ux}^T K
\end{array}
$$

Once these terms are computed, a forward pass computes a new trajectory:

$$
\begin{array}{lll}
\hat{x}_0 &= x_0\\
\hat{u}_t &= u_t + k_t + K_t(\hat{x}_t - x_t)\\
\hat{x}_{t+1} &= f(\hat{x}_t, \hat{u}_t)
\end{array}
$$

The basic flow of the algorithm is:

1. Initialize with initial state $\mathbf{x}_0$ and initial control sequence $\mathbf{U} = \{\mathbf{u}_{0}, \mathbf{u}_{1}, \dots, \mathbf{u}_{N-1}\}$.
2. Do a forward pass using the non-linear dynamics, i.e. simulate the system using $(\mathbf{x}_0, \mathbf{U})$ to get the trajectory through state space, $\mathbf{X}$, that results from applying the control sequence $\mathbf{X}$ starting in $\mathbf{x}_0$.
3. Do a backward pass, estimate the value function and dynamics for each $(\mathbf{x}, \mathbf{u})$ in the state-space and control signal trajectories.
4. Calculate an updated control signal $\hat{\mathbf{U}}$ and evaluate cost of trajectory resulting from $(\mathbf{x}_0, \hat{\mathbf{U}})$.
   
   1. If $|(\textrm{cost}(x_0, \hat{\textbf{U}}) - \textrm{cost}(x_0, \textbf{U})| < \textrm{threshold},$ then we've converged and exit.
   2. If $\textrm{cost}(x_0, \hat{\textbf{U}}) < \textrm{cost}(x_0, \textbf{U}),$ then set $\textbf{U} = \hat{\textbf{U}},$ and change the update size to be more aggressive. Go back to step 2.
   3. If $\textrm{cost}(x_0, \hat{\textbf{U}}) \geq \textrm{cost}(x_0, \textbf{U}),$ change the update size to be more modest. Go back to step 3.

## Differential Dynamic Programming

Differential Dynamic Programming (DDP) is almost identical to iLQR but unlike the latter, it expands the system's dynamics to the second order i.e. for a given a discrete-time non-linear system with state-space representation: 

$$
\mathbf{x}_{t+1} = f(\mathbf{x}_t, \mathbf{u}_t)
$$

we approximate it with a **second-order** Taylor-series expansion about nominal trajectories
$\mathbf{X} = \{\mathbf{x}_0, \dots, \mathbf{x}_N\}, \mathbf{U} = \{ \mathbf{u}_0, \dots, \mathbf{u}_N \}$:

$$
\begin{array}{lll}
\mathbf{x}_{t+1} + \delta\mathbf{x}_{t+1} &= f(\mathbf{x}_t + \delta \mathbf{x}_t, \mathbf{u}_t + \delta \mathbf{u}_t) \\ 
& \approx f(\mathbf{x}_t, \mathbf{u}_t)
+ \frac{\partial f}{\partial \mathbf{x}}|_{\mathbf{x}_t, \mathbf{u}_t}\delta\mathbf{x}_t
+ \frac{\partial f}{\partial \mathbf{u}}|_{\mathbf{x}_t, \mathbf{u}_t}\delta\mathbf{u}_t
+ \frac{1}{2} \left(
\frac{\partial^2 f}{\partial \mathbf{x}^2}|_{\mathbf{x}_t, \mathbf{u}_t}\delta\mathbf{x}_t^2
+ 2 \frac{\partial^2 f}{\partial \mathbf{x}\mathbf{u}}|_{\mathbf{x}_t, \mathbf{u}_t}\delta\mathbf{x}_t\delta\mathbf{u}_t
+ \frac{\partial^2 f}{\partial \mathbf{u}^2}|_{\mathbf{x}_t, \mathbf{u}_t}\delta\mathbf{u}_t^2
\right)
\\
\delta\mathbf{x}_{t+1} &= f_x\delta\mathbf{x}_t + f_u\delta\mathbf{u}_t + \frac{1}{2} \left(
f_{xx} \delta\mathbf{x}_t^2 + 2 f_{xu} \delta\mathbf{x}_t \delta\mathbf{u}_t + f_{uu} \delta\mathbf{u}_t^2
\right)
\end{array}
$$

Which leads to a different expansion of the $Q$-function:

$$
\begin{array}{lll}
Q_x &= l_x + f_x^T s_{t+1} \\
Q_u &= l_u + f_u^T s_{t+1} \\
Q_{xx} &= l_{xx} + f_x^T S_{t+1} f_x + S_{t+1} \cdot f_{xx} \\
Q_{uu} &= l_{uu} + B_t^T S_{t+1} B_t + S_{t+1} \cdot f_{xu} \\
Q_{ux} &= l_{ux} + B_t^T S_{t+1} A_t + S_{t+1} \cdot f_{uu} 
\end{array}
$$

The difference with iLQR is that the last term in the last 3 equations is the product of a vector with a tensor. 

The optimal control modification $\delta\mathbf{u}^∗$ for some state perturbation $\delta\mathbf{x}$, is obtained by minimizing the quadratic model:

$$
\delta\mathbf{u}^∗(\delta\mathbf{x}) = \displaystyle\arg\min_{\delta\mathbf{u}} Q(\delta\mathbf{x}, \delta\mathbf{u}) = k + K\delta\mathbf{x}
$$

This is a locally-linear feedback policy with:

$$
k := -Q_{uu}^{-1}Q_u\\
K := -Q_{uu}^{-1}Q_{ux}
$$

Plugging this back into the expansion of $Q$, a quadratic model of $V$ is obtained. After simplification it is:

$$
\begin{array}{lll}
\Delta V &= -\frac{1}{2} k^T Q_{uu} k\\
s &= Q_x - K^T Q_{uu} k\\
S &= Q_{xx} - K^T Q_{uu} K
\end{array}
$$

### Mass-Spring-Damper

[do-mpc](https://www.do-mpc.com/en/latest/) uses [CasADi](https://web.casadi.org/python-api/) (an open-source tool for nonlinear optimization and algorithmic differentiation) for the modeling part and for the different cost functions. Here's a table of useful operators:

| Operator | Description |Equation |
| --- | --- | --- |
| [casadi.sumsqr(x)](https://web.casadi.org/python-api/#casadi.casadi.sumsqr) | Squared-sum | $\sum x_i^2$ |
| [casadi.norm_2(x)](https://web.casadi.org/python-api/#casadi.casadi.norm_2) | $L_2$-norm | $\sqrt{\sum x_i^2}$ |
| [casadi.norm_1(x)](https://web.casadi.org/python-api/#casadi.casadi.norm_1) | $L_1$-norm | $\sum |x_i|$ |
| [casadi.bilin(A, x)](https://web.casadi.org/python-api/#casadi.casadi.bilin) | Quadratic Form | $x^T A x$ |
| [casadi.bilin(A, x, y)](https://web.casadi.org/python-api/#casadi.casadi.bilin) | Bilinear Form | $x^T A y$ |

#### Controller

First, we create an instance of the MPC class is generated with the Mass-Spring-Damper prediction model defined above.

In [ ]:
mpc = do_mpc.controller.MPC(mass_spring_damper)

We choose the finite prediction horizon `n_horizon = 20`, the time step `t_step = 0.04s` to be the same as the environment's time step. We also set the parameters of the applied discretization scheme orthogonal collocation.

In [ ]:
env = create_mass_spring_damper_environment()
mpc_params = {
    "n_horizon": 20,
    "t_step": env.dt,
    "state_discretization": "collocation",
    "collocation_type": "radau",
    "collocation_deg": 3,
    "collocation_ni": 1,
    "store_full_solution": True,
    # Use MA27 linear solver in ipopt for faster calculations:
    "nlpsol_opts": {"ipopt.linear_solver": "mumps"},
}
mpc.set_param(**mpc_params)

#### Objective

The control objective is to move the mass to a desired position (`0.1`) and keep it there.

In [ ]:
xss = np.array([0.1, 0.0])
distance_cost = 100 * casadi.norm_2(mass_spring_damper.x.cat - xss)
terminal_cost = distance_cost
stage_cost = distance_cost
print(f"{stage_cost=}")
print(f"{terminal_cost=}")
mpc.set_objective(mterm=terminal_cost, lterm=stage_cost)

We also restrict the input force.

In [ ]:
force_penalty = 1e-2
mpc.set_rterm(force=force_penalty)

#### Constraints

We apply constraints on the force. In this case, there is only an upper and lower bounds for the force.

In [ ]:
# lower and upper bounds of the input
u_max = 20
mpc.bounds["lower", "_u", "force"] = -u_max
mpc.bounds["upper", "_u", "force"] = u_max

#### Setup

We can now setup the controller.

In [ ]:
mpc.setup()

#### Simulation

We set the initial state and simulate the closed-loop for 100 steps.

In [ ]:
%%capture
mpc.reset_history()
mass_spring_damper_simulator.reset_history()
x0 = np.zeros((2, 1))
mass_spring_damper_simulator.x0 = x0
mpc.x0 = x0
mpc.set_initial_guess()
for k in range(100):
    u0 = mpc.make_step(x0)
    x0 = mass_spring_damper_simulator.make_step(u0)

In [ ]:
animate_mass_spring_damper_simulation(mpc.data)

#### Evaluation

Finally, we evaluate the controller on the actual environment.

In [ ]:
class MPCController:
    def __init__(self, mpc: do_mpc.controller.MPC) -> None:
        self.mpc = mpc
        self.mpc.reset_history()
        self.mpc.x0 = np.zeros(2)
        self.mpc.set_initial_guess()

    def act(self, observation: NDArray) -> NDArray:
        return mpc.make_step(observation.reshape(-1, 1)).ravel()

In [ ]:
%%capture
max_steps = 100
env = create_mass_spring_damper_environment(
    render_mode=render_mode, max_steps=max_steps
)
controller = MPCController(mpc)
results = simulate_environment(env, max_steps=max_steps, controller=controller)

In [ ]:
show_video(results.frames, fps=1 / env.dt)

In [ ]:
animate_mass_spring_damper_simulation(mpc.data)

## Exercise

- Design an MPC controller for the linearized inverted pendulum.
- For each case, try different cost functions:
  - $\sum \theta^2$
  - $\sum |\theta|$
  - $E_{\text{kinetic}} - E_{\text{potential}}$

## Solution

#### Controller

In [ ]:
mpc = do_mpc.controller.MPC(inverted_pendulum_lin)

In [ ]:
env = create_inverted_pendulum_environment()
mpc_params = {
    "n_horizon": 50,
    "t_step": env.dt,
    "state_discretization": "collocation",
    "collocation_type": "radau",
    "collocation_deg": 3,
    "collocation_ni": 1,
    "store_full_solution": True,
    # Use MA27 linear solver in ipopt for faster calculations:
    "nlpsol_opts": {"ipopt.linear_solver": "mumps"},
}
mpc.set_param(**mpc_params)

#### Objective

In [ ]:
xss = np.array([0.0, 0.0])
distance_cost = casadi.bilin(np.diag([100, 1]), inverted_pendulum_lin.x.cat - xss)
terminal_cost = distance_cost
stage_cost = distance_cost
print(f"{stage_cost=}")
print(f"{terminal_cost=}")
mpc.set_objective(mterm=terminal_cost, lterm=stage_cost)

In [ ]:
force_penalty = 1e-4
mpc.set_rterm(force=force_penalty)

#### Constraints

In [ ]:
# lower and upper bounds of the input
u_max = 3
mpc.bounds["lower", "_u", "force"] = -u_max
mpc.bounds["upper", "_u", "force"] = u_max

#### Setup

In [ ]:
mpc.setup()

#### Simulation

In [ ]:
%%capture
mpc.reset_history()
inverted_pendulum_lin_simulator.reset_history()
x0 = np.zeros((2, 1))
x0[0] = 0.01
inverted_pendulum_lin_simulator.x0 = x0
mpc.x0 = x0
mpc.set_initial_guess()
for k in range(100):
    u0 = mpc.make_step(x0)
    x0 = inverted_pendulum_lin_simulator.make_step(u0)

In [ ]:
animate_inverted_pendulum_simulation(mpc.data)

#### Evaluation

In [ ]:
class MPCController:
    def __init__(self, mpc: do_mpc.controller.MPC) -> None:
        self.mpc = mpc
        self.mpc.reset_history()
        self.mpc.x0 = np.zeros(2)
        self.mpc.set_initial_guess()

    def act(self, observation: NDArray) -> NDArray:
        return mpc.make_step(observation[[1, 3]].reshape(-1, 1)).ravel()

In [ ]:
%%capture
max_steps = 500
env = create_inverted_pendulum_environment(render_mode=render_mode, max_steps=max_steps)
controller = MPCController(mpc)
results = simulate_environment(env, max_steps=max_steps, controller=controller)

In [ ]:
show_video(results.frames, fps=1 / env.dt)

In [ ]:
animate_inverted_pendulum_simulation(mpc.data)

## Exercise

- Design an MPC Controller for the non-linear inverted pendulum system for two different cases:
  1. Cart at origin and upright Pendulm: Set the reference for the cart position to the origin.
  2. Pendulum Swing-up: Set the initial angle to to $-\pi$ i.e. start with the pendulum at the bottom.
  3. Same as 2 but set the reference for the cart position to the origin.
  4. Make the pendulum rotate as fast as possible.
  
> **Note 1** Use the following to create the environment with initial angle set to -np.pi and cutoff angle to np.inf for second > case.
> ```python
> env = create_inverted_pendulum_environment(cutoff_angle=np.inf, initial_angle=-np.pi)
> ```

> **Note 2**: You can access the inverted pendulum's kinetic, respectively potential, energy using
> `inverted_pendulum.aux["E_kinetic"]`, respectively `inverted_pendulum.aux["E_potential"]`

## Solution

### Swing-up

#### Controller

In [ ]:
mpc = do_mpc.controller.MPC(inverted_pendulum)

In [ ]:
env = create_inverted_pendulum_environment()
mpc_params = {
    "n_horizon": 100,
    "t_step": env.dt,
    "state_discretization": "collocation",
    "collocation_type": "radau",
    "collocation_deg": 3,
    "collocation_ni": 1,
    "store_full_solution": True,
    # Use MA27 linear solver in ipopt for faster calculations:
    "nlpsol_opts": {"ipopt.linear_solver": "mumps"},
}
mpc.set_param(**mpc_params)

#### Objective

In [ ]:
energy_cost = inverted_pendulum.aux["E_kinetic"] - inverted_pendulum.aux["E_potential"]
terminal_cost = energy_cost
stage_cost = energy_cost
print(f"{stage_cost=}")
print(f"{terminal_cost=}")
mpc.set_objective(mterm=terminal_cost, lterm=stage_cost)

In [ ]:
force_penalty = 0.1
mpc.set_rterm(force=force_penalty)

#### Constraints

In [ ]:
# lower and upper bounds of the position
x_max = 1
mpc.bounds["lower", "_x", "position"] = -x_max
mpc.bounds["upper", "_x", "position"] = x_max
# lower and upper bounds of the input
u_max = 3
mpc.bounds["lower", "_u", "force"] = -u_max
mpc.bounds["upper", "_u", "force"] = u_max

#### Setup

In [ ]:
mpc.setup()

#### Simulation

In [ ]:
%%capture
mpc.reset_history()
inverted_pendulum_simulator.reset_history()
x0 = np.array([0.0, -0.99 * np.pi, 0.0, 0.0])
inverted_pendulum_simulator.x0 = x0
mpc.x0 = x0
mpc.set_initial_guess()
for k in range(100):
    u0 = mpc.make_step(x0)
    x0 = inverted_pendulum_simulator.make_step(u0)

In [ ]:
animate_full_inverted_pendulum_simulation(mpc.data)

#### Evaluation

In [ ]:
class MPCController:
    def __init__(self, mpc: do_mpc.controller.MPC) -> None:
        self.mpc = mpc
        self.mpc.reset_history()
        self.mpc.x0 = np.array([0.0, -0.99 * np.pi, 0.0, 0.0])
        self.mpc.set_initial_guess()

    def act(self, observation: NDArray) -> NDArray:
        return mpc.make_step(observation.reshape(-1, 1)).ravel()

In [ ]:
%%capture
max_steps = 500
env = create_inverted_pendulum_environment(
    render_mode=render_mode,
    max_steps=max_steps,
    cutoff_angle=np.inf,
    initial_angle=-np.pi,
)
controller = MPCController(mpc)
results = simulate_environment(env, max_steps=max_steps, controller=controller)

In [ ]:
show_video(results.frames, fps=1 / env.dt)

In [ ]:
animate_full_inverted_pendulum_simulation(mpc.data)

# MPC and AlphaZero

AlphaZero is a computer program developed by artificial intelligence research company DeepMind to master the games of chess, shogi and go.

It is built from three core pieces:

- Value Function (Neural Network) to estimate the optimal cost-to-go for any given state.
- Policy (Neural Network) to determine the action to take at a given state.
- Monte Carlo Tree Search to simulate and search for the best plan.

```{figure} _static/images/50_alphazero_online_play.png
:width: 60%
:align: center
Illustration of an on-line player such as the one used in AlphaGo,
AlphaZero, and Tesauro’s backgammon program. At a given position,
it generates a lookahead tree of multiple moves up to some depth, then runs
the off-line obtained player for some more moves, and evaluates the effect of the
remaining moves by using the position evaluator of the off-line player.
```

In AlphaZero, the policy and value networks are trained off-line and an approximate version of the fundamental DP algorithm of policy iteration. A separate on-line player is used to select moves, based on multistep lookahead minimization and a terminal position evaluator that was trained using experience with the off-line player.

This approach performs better than using the off-line policy directly because of the long lookahead minimization, which corrects for the inevitable imperfections of the neural network-trained off-line
player, and position evaluator/terminal cost approximation.

In model predictive control (MPC), there is no off-line training and we use the system's model for the on-line rollout. The control interval is equivalent to the number of steps in lookahead minimization, while the prediction interval is equivalent to the total number of steps in lookahead minimization and truncated rollout.

## Monte Carlo Tree Search

Monte Carlo Tree Search (MCTS) is a heuristic search algorithm which uses stochastic simulations for decision processes, most notably those employed in software that plays board games. In that context MCTS is used to solve the game tree.

MCTS was combined with neural networks in 2016 and has been used in multiple board games like Chess, Shogi, Checkers, Backgammon, Contract Bridge, Go, Scrabble, and Clobber as well as in turn-based-strategy video games (such as Total War: Rome II's implementation in the high level campaign AI). 

The method especially took off due its effectiveness in computer Go and is still being used in DeepMind’s AlphaGoZero the most advanced Go AI to date.

MCTS consists of 4 steps that are repeated until some condition is met:

1. **Selection**: The selection phase traverses the tree level by level, each time
   selecting a node based on stored statistics like "number of visits"
   or "total reward". The rule by which the algorithm selects is called the tree policy.
   Selection stops when a node is reached that is not fully explored yet, i.e.
   not all possible moves have been expanded to new nodes yet.
2. **Expansion**: The expansion step consists of adding one or multiple new 
   child nodes to the final selected node.

3. **Rollout**: A rollout is initiated from each of these added nodes.
   Depending on the mean depth of the tree, rollout is done until the end of the game or
   as deep as possible before performing an evaluation. When dealing with
   terminal states in games the evaluation is done based on the win condition
   and usually yields 0 (loss), 0.5 (draw) or 1 (win).
   In the original algorithm the rollout is performed at random, but in general
   the so-called default policy can be enhanced by heuristics. For example,
   in the case of AlphaGoZero the rollout step was completely replaced by a
   neural network evaluation function.
4. **Backup**: The outcome of the rollout step is backed up to the nodes involved 
   in the selection phase by updating their respective statistics

```{figure} _static/images/50_monte_carlo_tree_search.svg
:width: 80%
```

# Summary

- Planning plays an important role in control, decision making and reinforcement learning.
- Optimal Control is the branch of Control Theory concerned with optimizing a cost function through the use of open-loop planning.
- The Linear Quadratic Regulator (LQR) is an optimal feedback control law that optimizes a quadratic cost function for LTI systems.
- Model Predictive Control (MPC) is a control scheme that uses planning in a receding horizon to determine the best action to take at each time step.
- MPC uses the system's model for the rollout and can approached in many different ways such as iLQR and DDP.
- AlphaZero uses off-line learning and on-line planning to tackle different types of games. Unlike MPC it uses neural networks to determine the next best action and to approximate the cost-to-go. 

:::{figure} ./_static/images/aai-institute-cover.png
:width: 90%
:align: center
---
name: aai-institute
---
:::

# References

- [<b id="slotine_applied_1991">[Slotine 1991]</b>](#slotine_applied_1991-back) [Applied nonlinear control.](https://www.academia.edu/download/33582713/Applied_Nonlinear_Control_Slotine.pdf) Slotine, Jean-Jacques E., and Weiping Li. Vol. 199, no. 1. Englewood Cliffs, NJ: Prentice hall. 1991.

- [<b id="grune_nonlinear_2017">[Lars Nonlinear 2017]</b>](#grune_nonlinear_2017-back) [Nonlinear model predictive control.](https://link.springer.com/chapter/10.1007/978-3-319-46024-6_3) Grüne, Lars, Jürgen Pannek, Lars Grüne, and Jürgen Pannek. Springer International Publishing, 2017.

- [<b id="biral_notes_2016">[Biral Notes 2016]</b>](#biral_notes_2016-back) [Notes on numerical methods for solving optimal control problems.](https://www.jstage.jst.go.jp/article/ieejjia/5/2/5_154/_article/-char/ja/) Biral, Francesco, Enrico Bertolazzi, and Paolo Bosetti. IEEJ Journal of Industry Applications 5, no. 2 (2016): 154-166.

- [<b id="bertsekas_lessons_2022">[Bertsektas, Dimitri P, 2022]</b>](#bertsekas_lessons_2022-back) [Lessons from AlphaZero for Optimal, Model Predictive, and Adaptive Control](http://web.mit.edu/dimitrib/www/LessonsfromAlphazero.pdf) - Dimitri P. Bertsektas. 2022.

- [<b id="spencer_optimal_2023">[Spencer et al. 2023]</b>](#spencer_optimal_2023-back) [AA 203: Optimal and Learning-Based Control.](https://stanfordasl.github.io/aa203/sp2223/) Optimal control solution techniques for systems with known and unknown dynamics. 2023.

- [<b id="tedrake_underactuated_2023">[Russ Tedrake, 2023]</b>](#tedrake_underactuated_2023-back) [Underactuated Robotics: Algorithms for Walking, Running, Swimming, Flying, and Manipulation (Course Notes for MIT 6.832)](http://underactuated.mit.edu/index.html) Russ Tedrake. Downloaded on 24.10.2023.